## EDA

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import geopy
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

## Loading Data

In [2]:
train = pd.read_csv('../../assets/train.csv')
test = pd.read_csv('../../assets/test.csv')
spray = pd.read_csv('../../assets/spray.csv')
weather = pd.read_csv('../../assets/weather.csv')

## Combine Train and Test to Ensure Column Order

In [3]:
all_data = pd.concat([train, test], sort=False)
all_data.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Id
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1.0,0.0,NaN
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1.0,0.0,NaN
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1.0,0.0,NaN
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1.0,0.0,NaN
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4.0,0.0,NaN


## Looking for Missing Data

In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


**Missing values appear to have already been filled with 'M'.**  
We're going to use `value_counts()` to look at individual columns to determine whether or not they contain useful information.

In [5]:
weather['SnowFall'].value_counts()

M      1472
0.0    1459
  T      12
0.1       1
Name: SnowFall, dtype: int64

In [6]:
weather['Water1'].value_counts()

M    2944
Name: Water1, dtype: int64

In [7]:
weather['Depth'].value_counts()

0    1472
M    1472
Name: Depth, dtype: int64

**Drop columns that don't contain useful information.**

In [8]:
useless_features = ['CodeSum', 'SnowFall', 'Water1', 'Depth'] 
weather.drop(labels=useless_features, axis=1, inplace=True)

In [9]:
weather = weather.replace('M', 0.0)
weather = weather.replace('T', 0.005)
weather = weather.replace(' T', 0.005)
weather = weather.replace('  T', 0.005)

## Fixing dtypes for Weather

In [10]:
weather['Tavg'] = weather['Tavg'].astype('int64')
weather['Depart'] = weather['Depart'].astype('int64')
weather['WetBulb'] = weather['WetBulb'].astype('int64')
weather['Heat'] = weather['Heat'].astype('int64')
weather['Cool'] = weather['Cool'].astype('int64')
weather['PrecipTotal'] = weather['PrecipTotal'].astype('float64')
weather['StnPressure'] = weather['StnPressure'].astype('float64')
weather['SeaLevel'] = weather['SeaLevel'].astype('float64')
weather['ResultSpeed'] = weather['ResultSpeed'].astype('float64')
weather['AvgSpeed'] = weather['AvgSpeed'].astype('float64')

In [11]:
station_1_mask = weather['Station'] == 1
station_1 = weather[station_1_mask]
station_1.drop('Station', axis=1, inplace=True)

/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
station_1.head()

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,0.000,29.10,29.82,1.7,27,9.2
2,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,0.000,29.38,30.09,13.0,4,13.4
4,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,0.000,29.39,30.12,11.7,7,11.9
6,2007-05-04,66,49,58,4,41,50,7,0,0444,1852,0.005,29.31,30.05,10.4,8,10.8
8,2007-05-05,66,53,60,5,38,49,5,0,0443,1853,0.005,29.40,30.10,11.7,7,12.0


In [13]:
station_2_mask = weather['Station'] == 2
station_2 = weather[station_2_mask]
station_2.drop('Station', axis=1, inplace=True)

/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [14]:
station_2.head()

,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
1,2007-05-01,84,52,68,0,51,57,0,3,-,-,0.000,29.18,29.82,2.7,25,9.6
3,2007-05-02,60,43,52,0,42,47,13,0,-,-,0.000,29.44,30.08,13.3,2,13.4
5,2007-05-03,67,48,58,0,40,50,7,0,-,-,0.000,29.46,30.12,12.9,6,13.2
7,2007-05-04,78,51,0,0,42,50,0,0,-,-,0.000,29.36,30.04,10.1,7,10.4
9,2007-05-05,66,54,60,0,39,50,5,0,-,-,0.005,29.46,30.09,11.2,7,11.5


Drop empty colums from station 2 since they would be mostly the same if they were imputed using station 1 anyway.

In [15]:
station_1.reset_index(drop=True)
station_2.reset_index(drop=True)
station_2.drop(labels=['Depart', 'Sunrise', 'Sunset'], axis=1, inplace=True)

/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [16]:
station_2.head()

,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
1,2007-05-01,84,52,68,51,57,0,3,0.000,29.18,29.82,2.7,25,9.6
3,2007-05-02,60,43,52,42,47,13,0,0.000,29.44,30.08,13.3,2,13.4
5,2007-05-03,67,48,58,40,50,7,0,0.000,29.46,30.12,12.9,6,13.2
7,2007-05-04,78,51,0,42,50,0,0,0.000,29.36,30.04,10.1,7,10.4
9,2007-05-05,66,54,60,39,50,5,0,0.005,29.46,30.09,11.2,7,11.5


In [17]:
weather = station_1.merge(station_2, on='Date', suffixes=('_s1', '_s2'))

In [18]:
weather['Date'] = pd.to_datetime(weather['Date'])

In [19]:
weather['Sunrise'] = weather['Sunrise'].map(lambda x: f'{x[:2]}:{x[2:]}')
weather['Sunrise'] = pd.to_datetime(weather['Sunrise'], format='%H:%M')

In [20]:
weather.head()

,Date,Tmax_s1,Tmin_s1,Tavg_s1,Depart,DewPoint_s1,WetBulb_s1,Heat_s1,Cool_s1,Sunrise,...,DewPoint_s2,WetBulb_s2,Heat_s2,Cool_s2,PrecipTotal_s2,StnPressure_s2,SeaLevel_s2,ResultSpeed_s2,ResultDir_s2,AvgSpeed_s2
0,2007-05-01,83,50,67,14,51,56,0,2,1900-01-01 04:48:00,...,51,57,0,3,0.000,29.18,29.82,2.7,25,9.6
1,2007-05-02,59,42,51,-3,42,47,14,0,1900-01-01 04:47:00,...,42,47,13,0,0.000,29.44,30.08,13.3,2,13.4
2,2007-05-03,66,46,56,2,40,48,9,0,1900-01-01 04:46:00,...,40,50,7,0,0.000,29.46,30.12,12.9,6,13.2
3,2007-05-04,66,49,58,4,41,50,7,0,1900-01-01 04:44:00,...,42,50,0,0,0.000,29.36,30.04,10.1,7,10.4
4,2007-05-05,66,53,60,5,38,49,5,0,1900-01-01 04:43:00,...,39,50,5,0,0.005,29.46,30.09,11.2,7,11.5


For whatever reason some of the sunset times have 60 listed for the minutes. We'll apply the following function to them to increment the hour whenever we find a time with 60 for the minutes.

In [21]:
def right_time(time):
    if time[-2:] == '60':
        return f'{int(time[:2]) + 1}:00:00'
    else:
        return f'{time[:2]}:{time[2:]}:00'

In [22]:
weather['Sunset'] = weather['Sunset'].apply(right_time)
weather['SunsetTime'] = weather['Date'].astype(str) + ' ' + weather['Sunset']
weather['Sunset'] = pd.to_datetime(weather['SunsetTime'])

In [23]:
weather.drop('SunsetTime', axis=1, inplace=True)

In [24]:
weather['SunsetHour'] = weather['Sunset'].dt.hour
weather['SunsetMinute'] = weather['Sunset'].dt.minute
weather['SunriseHour'] = weather['Sunrise'].dt.hour
weather['SunriseMinute'] = weather['Sunrise'].dt.minute

In [25]:
weather.drop(['Sunrise', 'Sunset'], axis=1, inplace=True)

In [26]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 0 to 1471
Data columns (total 32 columns):
Date              1472 non-null datetime64[ns]
Tmax_s1           1472 non-null int64
Tmin_s1           1472 non-null int64
Tavg_s1           1472 non-null int64
Depart            1472 non-null int64
DewPoint_s1       1472 non-null int64
WetBulb_s1        1472 non-null int64
Heat_s1           1472 non-null int64
Cool_s1           1472 non-null int64
PrecipTotal_s1    1472 non-null float64
StnPressure_s1    1472 non-null float64
SeaLevel_s1       1472 non-null float64
ResultSpeed_s1    1472 non-null float64
ResultDir_s1      1472 non-null int64
AvgSpeed_s1       1472 non-null float64
Tmax_s2           1472 non-null int64
Tmin_s2           1472 non-null int64
Tavg_s2           1472 non-null int64
DewPoint_s2       1472 non-null int64
WetBulb_s2        1472 non-null int64
Heat_s2           1472 non-null int64
Cool_s2           1472 non-null int64
PrecipTotal_s2    1472 non-null float6

In [27]:
weather.set_index('Date', inplace=True)

## Merge Weather to All Data

In [28]:
all_data['Date'] = pd.to_datetime(all_data['Date'])
all_data['Month'] = all_data['Date'].dt.month
all_data['Weekday'] = all_data['Date'].dt.weekday
all_data['Week'] = all_data['Date'].dt.week
all_data['Year'] = all_data['Date'].dt.year

In [29]:
all_data.set_index('Date', inplace=True)

In [30]:
all_data = all_data.merge(weather, on='Date')

In [31]:
all_data[['Month', 'Weekday', 'Week']].head()

,Month,Weekday,Week
Date,,,
2007-05-29,5,1,22
2007-05-29,5,1,22
2007-05-29,5,1,22
2007-05-29,5,1,22
2007-05-29,5,1,22


In [32]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 126799 entries, 2007-05-29 to 2014-10-02
Data columns (total 47 columns):
Address                   126799 non-null object
Species                   126799 non-null object
Block                     126799 non-null int64
Street                    126799 non-null object
Trap                      126799 non-null object
AddressNumberAndStreet    126799 non-null object
Latitude                  126799 non-null float64
Longitude                 126799 non-null float64
AddressAccuracy           126799 non-null int64
NumMosquitos              10506 non-null float64
WnvPresent                10506 non-null float64
Id                        116293 non-null float64
Month                     126799 non-null int64
Weekday                   126799 non-null int64
Week                      126799 non-null int64
Year                      126799 non-null int64
Tmax_s1                   126799 non-null int64
Tmin_s1                   126799 non-null in

## Separate Out Train and Test

In [33]:
train = all_data[all_data['Id'].isnull()]
train.shape

(10506, 47)

In [34]:
test = all_data[pd.notnull(all_data['Id'])]
test.shape

(116293, 47)

## Correlation

In [35]:
train.corr()['WnvPresent'].sort_values()[:-2]

SunsetHour        -0.071959
Longitude         -0.060345
Heat_s1           -0.056813
Heat_s2           -0.052008
ResultSpeed_s1    -0.051177
ResultSpeed_s2    -0.040141
SunriseMinute     -0.035951
AvgSpeed_s1       -0.029958
AvgSpeed_s2       -0.028088
PrecipTotal_s1    -0.021426
ResultDir_s2      -0.007983
Block              0.004877
SeaLevel_s1        0.005620
SeaLevel_s2        0.007322
ResultDir_s1       0.007922
AddressAccuracy    0.008064
StnPressure_s2     0.011917
Weekday            0.014968
StnPressure_s1     0.022482
Latitude           0.028697
PrecipTotal_s2     0.039332
SunsetMinute       0.044231
Tmax_s2            0.045755
Tmax_s1            0.050536
Year               0.050865
Depart             0.055436
Cool_s2            0.056728
Cool_s1            0.058953
Tavg_s2            0.062072
Tavg_s1            0.066043
Tmin_s1            0.072719
Tmin_s2            0.074459
WetBulb_s1         0.077242
WetBulb_s2         0.077728
DewPoint_s2        0.080873
DewPoint_s1        0

In [48]:
train['Date'] = train.index

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
train.reset_index(drop=True, inplace=True)

In [56]:
train.groupby(by=['Date', 'Trap']).NumMosquitos.sum()

Date        Trap
2007-05-29  T002     2.0
            T007     1.0
            T015     5.0
            T045     2.0
            T046     1.0
            T048     3.0
            T049     1.0
            T050     1.0
            T054     5.0
            T086     2.0
            T091     1.0
            T094     3.0
            T096     6.0
            T129     3.0
            T143     1.0
            T148     1.0
            T153     1.0
            T159     1.0
2007-06-05  T002     9.0
            T009     2.0
            T011     1.0
            T015     3.0
            T016     1.0
            T019     1.0
            T025     1.0
            T028     2.0
            T031     1.0
            T033     3.0
            T045     4.0
            T046     1.0
                    ... 
2013-09-26  T099     5.0
            T102     4.0
            T114     4.0
            T115     7.0
            T128    11.0
            T135     7.0
            T138    10.0
            T145     2.0
        

## Baseline

In [ ]:
len(train[train['WnvPresent'] == 1]) / len(train)

## Save Merged Data as a New CSV

In [ ]:
# train.to_csv('../data/new_train.csv')
# test.to_csv('../data/new_test.csv')